In [1]:
# Import findspark to read SPARK_HOME and HADOOP_HOME

import findspark
findspark.init()

In [2]:
# Import required library

from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
# Print Spark object ID

print(spark)

In [4]:
df = spark.read.csv("D://5115100117//Big Data//Pengenalan Apache Spark//Case_Data_from_San_Francisco_311__SF311.csv", header=True, inferSchema=True)

In [5]:
import pixiedust

Pixiedust database opened successfully


In [24]:
from pyspark.sql import SQLContext

df.createOrReplaceTempView("casedata")
# df.registerTempTable("casedata")
sanDF = SQLContext.sql("SELECT * FROM casedata WHERE point is not NULL")

TypeError: sql() missing 1 required positional argument: 'sqlQuery'